In [5]:
import pandas as pd
import numpy as np

In [6]:
# Load the dataset
data = pd.read_csv("new-bank-additional-full.csv")
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [7]:
data['default'].value_counts()

default
no         32588
unknown     8597
yes            3
Name: count, dtype: int64

In [8]:
data['default'] = data['default'].map({'no': 0, 'yes': 1, 'unknown': -1})

In [9]:
data['housing'].value_counts()

housing
yes        21576
no         18622
unknown      990
Name: count, dtype: int64

In [10]:
data['housing'] = data['housing'].map({'no': 0, 'yes': 1, 'unknown': -1})

In [11]:
data['loan'].value_counts()

loan
no         33950
yes         6248
unknown      990
Name: count, dtype: int64

In [12]:
data['loan'] = data['loan'].map({'no': 0, 'yes': 1, 'unknown': -1})

In [13]:
data['contact'].value_counts()

contact
cellular     26144
telephone    15044
Name: count, dtype: int64

In [14]:
data['contact'] = data['contact'].map({'cellular': 1, 'telephone': 0})

In [15]:
data['month'].value_counts()

month
may    13769
jul     7174
aug     6178
jun     5318
nov     4101
apr     2632
oct      718
sep      570
mar      546
dec      182
Name: count, dtype: int64

In [16]:
month_mapping = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
                 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}

data['month'] = data['month'].map(month_mapping)

In [17]:
data['day_of_week'].value_counts()

day_of_week
thu    8623
mon    8514
wed    8134
tue    8090
fri    7827
Name: count, dtype: int64

In [18]:
day_mapping = {'mon': 1, 'tue': 2, 'wed': 3, 'thu': 4, 'fri': 5}
data['day_of_week'] = data['day_of_week'].map(day_mapping)

In [19]:
data['poutcome'].value_counts()

poutcome
nonexistent    35563
failure         4252
success         1373
Name: count, dtype: int64

In [20]:
poutcome_mapping = {'nonexistent': -1, 'failure': 0, 'success': 1}
data['poutcome'] = data['poutcome'].map(poutcome_mapping)

In [21]:
data['education'].value_counts()

education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
unknown                 1731
illiterate                18
Name: count, dtype: int64

In [22]:
education_mapping = {
    'illiterate': 0,
    'basic.4y': 1,
    'basic.6y': 2,
    'basic.9y': 3,
    'high.school': 4,
    'professional.course': 5,
    'university.degree': 6,
    'unknown': 0  # Treat 'unknown' the same as 'illiterate'
}

data['education'] = data['education'].map(education_mapping)

In [23]:
data['marital'].value_counts()

marital
married     24928
single      11568
divorced     4612
unknown        80
Name: count, dtype: int64

In [24]:
data['job'].value_counts()

job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: count, dtype: int64

In [25]:
# Replace "unknown" with NaN
columns_to_consider = ["marital","job"]
data = data.replace("unknown", np.nan)

# Before dropping rows, print the number of rows before dropping
rows_before = len(data)
print(f"Rows before removing unknown values: {rows_before}")

# Drop rows with NaN in specified columns
data = data.dropna(subset=columns_to_consider)

# After dropping rows, print the number of rows after dropping
rows_after = len(data)
print(f"Rows after removing unknown values: {rows_after}")

# Print how many rows were dropped
rows_deleted = rows_before - rows_after
print(f"Rows dropped after removing unknown values: {rows_deleted}")

Rows before removing unknown values: 41188
Rows after removing unknown values: 40787
Rows dropped after removing unknown values: 401


In [26]:
# Drop the 'duration' column as it leaks information
data.drop(columns=["duration"], inplace=True)

In [27]:
# Encode target variable
data['y'] = data['y'].map({"yes": 1, "no": 0}).astype(int)

In [28]:
# One-Hot Encoding for 'marital' column (creating three new columns: 'single', 'married', 'divorced')

# Identify the position of the original 'marital' column
marital_index = data.columns.get_loc('marital')

data['marital_single'] = (data['marital'] == 'single').astype(int)
data['marital_married'] = (data['marital'] == 'married').astype(int)
data['marital_divorced'] = (data['marital'] == 'divorced').astype(int)

# Drop the original 'marital' column
data.drop(columns=['marital'], inplace=True)

# Insert the new columns at the original position of 'marital'
data.insert(marital_index, 'marital_single', data.pop('marital_single'))
data.insert(marital_index + 1, 'marital_married', data.pop('marital_married'))
data.insert(marital_index + 2, 'marital_divorced', data.pop('marital_divorced'))

# Verify the column order
print(data.columns)

Index(['age', 'job', 'marital_single', 'marital_married', 'marital_divorced',
       'education', 'default', 'housing', 'loan', 'contact', 'month',
       'day_of_week', 'campaign', 'pdays', 'previous', 'poutcome',
       'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m',
       'nr.employed', 'y'],
      dtype='object')


In [29]:
# Identify the position of the original 'job' column
job_index = data.columns.get_loc('job')

# Perform One-Hot Encoding for 'job' column
job_dummies = pd.get_dummies(data['job'], prefix='job', drop_first=True)

# Drop the original 'job' column
data.drop(columns=['job'], inplace=True)

# Convert boolean True/False in job_dummies to 1/0
job_dummies = job_dummies.astype(int)

# Insert the new job columns at the original position of 'job'
for i, col in enumerate(job_dummies.columns):
    data.insert(job_index + i, col, job_dummies[col])

# Verify the changes
print(data.head(10))


   age  job_blue-collar  job_entrepreneur  job_housemaid  job_management  \
0   56                0                 0              1               0   
1   57                0                 0              0               0   
2   37                0                 0              0               0   
3   40                0                 0              0               0   
4   56                0                 0              0               0   
5   45                0                 0              0               0   
6   59                0                 0              0               0   
7   41                1                 0              0               0   
8   24                0                 0              0               0   
9   25                0                 0              0               0   

   job_retired  job_self-employed  job_services  job_student  job_technician  \
0            0                  0             0            0               0   
1  

In [30]:
# Specify all columns to be converted to 'category', including one-hot encoded columns
categorical_columns = ['default', 'housing', 'loan', 'contact', 'month',
                       'day_of_week', 'poutcome', 'education',
                       'marital_single', 'marital_married', 'marital_divorced']

# Add the one-hot encoded job columns to the list
job_columns = [col for col in data.columns if col.startswith('job_')]
categorical_columns.extend(job_columns)

# Convert all specified columns to categorical dtype
data[categorical_columns] = data[categorical_columns].astype('category')

# Verify the changes
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 40787 entries, 0 to 41187
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   age                40787 non-null  int64   
 1   job_blue-collar    40787 non-null  category
 2   job_entrepreneur   40787 non-null  category
 3   job_housemaid      40787 non-null  category
 4   job_management     40787 non-null  category
 5   job_retired        40787 non-null  category
 6   job_self-employed  40787 non-null  category
 7   job_services       40787 non-null  category
 8   job_student        40787 non-null  category
 9   job_technician     40787 non-null  category
 10  job_unemployed     40787 non-null  category
 11  marital_single     40787 non-null  category
 12  marital_married    40787 non-null  category
 13  marital_divorced   40787 non-null  category
 14  education          40787 non-null  category
 15  default            40787 non-null  category
 16  housing  

In [31]:
# Save the dataset for Neural Networks
data.to_csv("processed_bank_data.csv", index=False)